In [3]:
# assignment1.py — Single-model solution (Holt-Winters with weekly seasonality)
# Produces variables required by the assignment: model, modelFit, pred

import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# ------------------------------
# Helpers
# ------------------------------
def _infer_datetime_col(df):
    """Return the name of the datetime column (case-insensitive).
    Falls back to the first column that can be parsed as datetime."""
    candidates = ["datetime", "time", "timestamp", "date", "ds"]
    lower_cols = {c.lower(): c for c in df.columns}
    for c in candidates:
        if c in lower_cols:
            return lower_cols[c]
    # Fallback: try first column that parses to datetime
    for c in df.columns:
        parsed = pd.to_datetime(df[c], errors="coerce", utc=False)
        if parsed.notna().mean() > 0.95:  # most rows parse cleanly
            return c
    # If nothing obvious, just use the first column
    return df.columns[0]

def _load_hourly_csv(path):
    """Load CSV, parse datetime, set hourly index, clean small gaps."""
    tmp = pd.read_csv(path)
    dt_col = _infer_datetime_col(tmp)
    tmp[dt_col] = pd.to_datetime(tmp[dt_col])
    tmp = tmp.sort_values(dt_col).set_index(dt_col)
    tmp = tmp.asfreq("H")  # ensure hourly frequency
    # Basic cleaning for the target column
    # Guess target a


In [5]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.varmax import VARMAX

# Load the training and test data from the correct directory
train = pd.read_csv('/content/assignment_data_train.csv')
test = pd.read_csv('/content/assignment_data_test.csv')

# Remove timestamp column (if present)
if 'Timestamp' in train.columns:
    train = train.drop(columns=['Timestamp'])
if 'Timestamp' in test.columns:
    test = test.drop(columns=['Timestamp'])

# Ensure all columns are numeric
train = train.apply(pd.to_numeric, errors='coerce')
test = test.apply(pd.to_numeric, errors='coerce')

# Identify and drop constant columns (columns with only one unique value)
constant_cols = [col for col in train.columns if train[col].nunique() == 1]
train = train.drop(columns=constant_cols)
test = test.drop(columns=constant_cols, errors='ignore')

# Drop missing values
train = train.dropna()
test = test.dropna()

# DIFFERENCING: Ensure stationarity
train_diff = train.diff().dropna()

# Fit the VARMA model (VARMAX in statsmodels)
model = VARMAX(train_diff, order=(1, 1), enforce_stationarity=False, enforce_invertibility=False)
modelFit = model.fit(disp=False)

# Forecast for 744 hours
pred_diff = modelFit.forecast(steps=744)

# Convert predictions back to original scale
last_values = train.iloc[-1]  # Get last known values before differencing
pred = pred_diff.cumsum() + last_values['trips']

# Convert predictions into DataFrame
pred_df = pd.DataFrame(pred, columns=['trips'])

# Save predictions
pred_df.to_csv('/content/predictions_varma.csv', index=False)


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an excep